# **Hilal properties scraping**

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Define base URL and headers
base_url = "https://hilalprp.com.om"
page_pattern = base_url + "/properties-search/page/{}/?status=for-sale"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Store extracted property information
records = {
    "Property_Name": [],
    "City": [],
    "Beds": [],
    "Baths": [],
    "Garage": [],
    "Cost": [],
    "Area": [],
    "Ad_Type": []
}

max_pages = 100

for page in range(1, max_pages + 1):
    current_url = page_pattern.format(page)
    print(f"Processing page {page}: {current_url}")

    try:
        res = requests.get(current_url, headers=headers)
        res.raise_for_status()
    except Exception as error:
        print(f"Page fetch failed: {error}")
        break

    html = BeautifulSoup(res.content, "html.parser")
    property_blocks = html.find_all("article", class_="rh_list_card")

    if not property_blocks:
        print("No data found. Ending loop.")
        break

    for block in property_blocks:
        # Title
        heading = block.find("h3")
        name = heading.get_text(strip=True) if heading else "Unknown"

        # Price
        price_tag = block.find("p", class_="price")
        cost = price_tag.text.replace("OMR", "").replace(",", "").strip() if price_tag else "N/A"

        # Listing category
        status = block.find("span", class_="status")
        listing = status.text.strip() if status else "For Sale"

        # Meta fields
        info = {"Beds": "N/A", "Baths": "N/A", "Garage": "N/A", "Area": "N/A"}
        metas = block.find_all("div", class_="rh_prop_card__meta")
        for meta in metas:
            key = meta.find("span", class_="rh_meta_titles")
            val = meta.find("span", class_="figure")
            if key and val:
                tag = key.text.lower()
                value = val.text.strip()
                if "bed" in tag:
                    info["Beds"] = value
                elif "bath" in tag:
                    info["Baths"] = value
                elif "garage" in tag:
                    info["Garage"] = value
                elif "area" in tag or "size" in tag:
                    info["Area"] = value

        # Detail page for city
        city = "N/A"
        detail = block.find("a", href=True)
        if detail:
            try:
                detail_page = requests.get(detail["href"], headers=headers)
                detail_soup = BeautifulSoup(detail_page.content, "html.parser")
                city_tag = detail_soup.find("a", href=lambda x: x and "/property-city/" in x)
                if city_tag:
                    city = city_tag.text.strip()
            except Exception as e:
                print(f"Detail fetch error: {e}")

        # Store record
        records["Property_Name"].append(name)
        records["City"].append(city)
        records["Beds"].append(info["Beds"])
        records["Baths"].append(info["Baths"])
        records["Garage"].append(info["Garage"])
        records["Cost"].append(cost)
        records["Area"].append(info["Area"])
        records["Ad_Type"].append(listing)

    time.sleep(1)  # Be polite to the server

# Export data
df = pd.DataFrame(records)
df.to_csv("hilal_properties_cleaned.csv", index=False)
print("Scraping finished. Data saved to 'hilal_properties_cleaned.csv'.")


Processing page 1: https://hilalprp.com.om/properties-search/page/1/?status=for-sale
Processing page 2: https://hilalprp.com.om/properties-search/page/2/?status=for-sale
Processing page 3: https://hilalprp.com.om/properties-search/page/3/?status=for-sale
Processing page 4: https://hilalprp.com.om/properties-search/page/4/?status=for-sale
Processing page 5: https://hilalprp.com.om/properties-search/page/5/?status=for-sale
Processing page 6: https://hilalprp.com.om/properties-search/page/6/?status=for-sale
Processing page 7: https://hilalprp.com.om/properties-search/page/7/?status=for-sale
Processing page 8: https://hilalprp.com.om/properties-search/page/8/?status=for-sale
No data found. Ending loop.
Scraping finished. Data saved to 'hilal_properties_cleaned.csv'.
